In [1]:
from datasets import load_dataset
dataset = load_dataset("tau/commonsense_qa")

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
base_model = "NousResearch/Llama-2-7b-chat-hf"
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)


# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/chowder/.local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/chowder/.local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/chowder/.local/lib/python3.8/site-packages/transform

In [4]:
pipe = pipeline(task="text-generation", model=base_model, tokenizer=base_model, max_new_tokens=10)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
#genTest = "The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? (A) ignore (B) enforce (C) authoritarian (D) yell at (E) avoid. Please select an answer."

#result = pipe(genTest)

#print(result)


In [6]:
import pickle

modelName = "Llama-2-7b-chat-hf"

fileName = modelName + "generatedMaxNewTokens10.pickle"


try:
    with open(fileName, 'rb') as file:
        generatedOutputs = pickle.load(file)
        print("loaded " + str(len(generatedOutputs)) + " rows")
except FileNotFoundError:
    print("File not found making new object")
    generatedOutputs = {}

loaded 12102 rows


In [9]:
import time
import re
answerPattern = r'[A-E]\)'

lastSave = time.time()
newAnswers = 0
for q in dataset["test"]:#"validation"
    if q["question"] not in generatedOutputs:
        prompt = q["question"]
        for i in range(len(q["choices"]["text"])):
            prompt += " (" + q["choices"]["label"][i] + ") " + q["choices"]["text"][i]
        prompt += "."
        #print(newAnswers % 10)
        print("generating result for the prompt: " + prompt)
        result = pipe(prompt)
        #print(result[0]['generated_text'])
        #print(len(result[0]['generated_text']))
        #print(len(prompt))
        #print(len(result) - len(prompt))
        genText = result[0]['generated_text'][len(prompt):]
        print("result is: " + genText)

        print("correct answer is " + q["answerKey"])

        potentialMatches = re.findall(answerPattern, genText)
        print(potentialMatches)
        if  len(potentialMatches) != 0:
            parsedAnswer = re.findall(answerPattern, genText)[0][0]
            print("parsed answer is " + parsedAnswer)
            generatedOutputs[q["question"]] = {"prompt" : prompt, "genText" : genText, "parsed" : parsedAnswer}
            newAnswers += 1
        else:
            #enter some infor indicating that it wasn't found
            generatedOutputs[q["question"]] = {"prompt" : prompt, "genText" : genText, "parsed" : "None"}
            print("not able to parse an answer###################################################################################################################################")
        print("Number of new answers: " + str(newAnswers))
    if newAnswers % 10 == 1:
        with open(fileName, 'wb') as file:
            # Serialize and save the object to the file
            print("saved " + str(newAnswers) + " new answers in " + str(time.time() - lastSave))
            pickle.dump(generatedOutputs, file)
        lastSave = time.time()


with open(fileName, 'wb') as file:
    # Serialize and save the object to the file
    print("saved " + str(newAnswers) + " new answers in " + str(time.time() - lastSave))
    pickle.dump(generatedOutputs, file)

saved 0 new answers in 0.06681013107299805


In [8]:
import re

testText = "asdf\n\n\nThe Answer is (E) Because I think it is not (A)."

answerPattern = r'\([A-E]\)'


print(re.findall(answerPattern, testText)[0][1])

E
